In [1]:
import numpy as np
import shutil
import os

import sys
sys.path.append("../utils/")
sys.path.append("../calculating_predictions")
sys.path.append("../evaluation/")
from calculate_predictions import calculate_predictions
from evaluate import evaluate
from run_evaluation import *
from argparse import Namespace
from catboost import CatBoostRegressor, CatBoostClassifier

%reload_ext autoreload
%autoreload 2

In [2]:
def get_stacking_metric(i):
    args = Namespace()
    args.verbose = True
    args.first_feature = 0
    args.last_feature = -1
    args.data_folder = "../../../data/best_features_days_data/"
    args.out_folder = "res"
    args.type = "regression"
    args.need_position_feature = True
    args.labels_to_substruct = None
    args.metric_folder = "metrics"
    args.model_constructor = lambda verbose: CatBoostRegressor(verbose=False)
    args.additional_features = {
        day: [
            os.path.join("features_models_with_pos", str(i), filename)
            for filename in os.listdir(os.path.join("features_models_with_pos", str(i)))
        ]
        for day in [3, 6]
    }
    args.validation_day = None
    args.train_days = [3]
    args.test_days = [6]
    
    def clear(folder):
        if os.path.exists(folder):
            shutil.rmtree(folder)
        os.mkdir(folder)
    
    clear(args.out_folder)
    clear(args.metric_folder)
    
    calculate_predictions(args)
    evaluate(args.out_folder, args.data_folder, args.metric_folder, argmax_positions)
    
    with open("metrics/metrics.txt") as handler:
        metric = float(next(handler).strip().split()[0])
        return metric

In [4]:
np.mean(metrics), np.std(metrics)

(0.6358130865122522, 0.013269665349249615)

In [8]:
0.013269665349249615 / np.sqrt(15)

0.0034262128604926256

In [3]:
%%time
metrics = [get_stacking_metric(i) for i in range(15)]

"2018-06-21 09:41:27.027802": preprocesing started
"2018-06-21 09:41:27.787595":     base features shape: (20203, 80)
"2018-06-21 09:41:28.801502":     28 features added
"2018-06-21 09:41:28.801634":     result shape: (20203, 108)
"2018-06-21 09:41:28.805018": train features shape: (20203, 108)
"2018-06-21 09:41:29.391713": preprocesing finished
"2018-06-21 09:41:29.391896": start training on days [3]
"2018-06-21 09:41:29.391929": using fit without validation
"2018-06-21 09:41:51.633562": built 1000 trees
"2018-06-21 09:41:53.780041":     base features shape: (247423, 80)
"2018-06-21 09:41:58.371784":     28 features added
"2018-06-21 09:41:58.371881":     result shape: (247423, 108)
"2018-06-21 09:41:58.372426": days to predict: [6]
"2018-06-21 09:41:58.372511": features shape: (247423, 108)
"2018-06-21 09:41:58.372582": start predicting on day 6
"2018-06-21 09:42:00.209862": predictions shape: (22493, 11)
"2018-06-21 09:42:00.210024": saveing results to res/train_3_test_6
"2018-06-21

"2018-06-21 09:46:21.357559": preprocesing started
"2018-06-21 09:46:22.116516":     base features shape: (20203, 80)
"2018-06-21 09:46:22.607319":     28 features added
"2018-06-21 09:46:22.607438":     result shape: (20203, 108)
"2018-06-21 09:46:22.608961": train features shape: (20203, 108)
"2018-06-21 09:46:23.201178": preprocesing finished
"2018-06-21 09:46:23.201259": start training on days [3]
"2018-06-21 09:46:23.201277": using fit without validation
"2018-06-21 09:46:46.874821": built 1000 trees
"2018-06-21 09:46:49.040058":     base features shape: (247423, 80)
"2018-06-21 09:46:53.426761":     28 features added
"2018-06-21 09:46:53.427001":     result shape: (247423, 108)
"2018-06-21 09:46:53.427841": days to predict: [6]
"2018-06-21 09:46:53.427916": features shape: (247423, 108)
"2018-06-21 09:46:53.427953": start predicting on day 6
"2018-06-21 09:46:55.379913": predictions shape: (22493, 11)
"2018-06-21 09:46:55.380059": saveing results to res/train_3_test_6
"2018-06-21

In [7]:
metrics

[0.6225470742075694,
 0.6399766834405218,
 0.6285494084457526,
 0.629198965100085,
 0.650858747287769,
 0.6578339449469228,
 0.6325487180627116,
 0.6277769965778034,
 0.6166481354176804,
 0.6376793776602243,
 0.6429636672419875,
 0.6195414785123247,
 0.6354835001445083,
 0.6306047000628527,
 0.6649849005750695]